# Analyze models predictions

Analysze the models predictions

In [ ]:
import sys
sys.path.append("./../")

In [ ]:
import os
import json
import glob

import numpy as np
import torch
from torch.utils.data import DataLoader

import methods
import models
import datasets
import transforms

import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
def get_predictions(model, dataset, N=10):
    dataloader = DataLoader(dataset, batch_size=1024, shuffle=False)

    # Get predictions from model
    ypreds = []
    with torch.no_grad():
        for x, y in dataloader:
            _ypreds, _ = model.sample_predictions(x, n=N)
            if ypreds:
                for i in range(N):
                    ypreds[i] = torch.cat((ypreds[i], _ypreds[i]), dim=0)
            else:
                for y2 in _ypreds:
                    # y2 is a tensor of shape (B, K)
                    ypreds.append(y2)

    # Convert to softmax score from log_softmax
    yprobs = [torch.exp(_ypreds) for _ypreds in ypreds]
    
    return yprobs

Edit `model_dir` variable to point to the model you want.

In [ ]:
model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/term-all-beta1010-20220402213949/"

# Default paths
config_json = os.path.join(model_dir, "config.json")
ckpt_file = glob.glob(model_dir + '/step=*.ckpt')[-1]

In [ ]:
config = json.load(open(config_json, 'r'))

MethodClass = getattr(methods, config['method'])
DatasetClass = getattr(datasets, config['dataset']) # Will automatically detect the dataset for testing
ModelClass = getattr(models, config['model'])
TransformClass = getattr(transforms, config['transform'])

In [ ]:
testset = DatasetClass(**config['ds_params'], split='test', transform=TransformClass())
K = testset.n_labels

In [ ]:
model = MethodClass.load_from_checkpoint(ckpt_file, model=ModelClass(K), strict=False)

## Get predictions from model

You can sample multiple predictions from the model for each realization of parameters by setting $N$. The returned
variable will be a list with $N$ entries, corresponding to each realizations. If you want average over all realization,
you can average over the list entries.

In [ ]:
yscores = get_predictions(model, testset, N=8) # Ask for 8 realizations of NN parameters

In [ ]:
len(yscores)

In [ ]:
yscores[0].shape